<a href="https://colab.research.google.com/github/ehsan-lari/pyro101/blob/main/pyro_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0. Setup

In [ ]:
!pip install -q pyro-ppl

In [ ]:
import pyro
import pyro.distributions as dist

import torch

In [ ]:
normal = dist.Normal(
    torch.tensor([0.0, 1.0, 2.0]),
    1.0)

In [ ]:
# torch.manual_seed(42)

normal.sample()

In [ ]:
normal2 = dist.Normal(0, 1)

In [ ]:
normal2.sample((1, 3, 2))

### 1. Introducing Models 1.0

In [ ]:
def model(mu = 15.0, sigma = 2.0):
  temp = pyro.sample('temp', dist.Normal(mu, sigma))
  return temp

print(model())

In [ ]:
pyro.render_model(model, render_distributions=True)

In [ ]:
def model2(mu = 15.0, sigma = 2.0):
  temp = pyro.sample('temp', dist.Normal(mu, sigma))
  sensor = pyro.sample('sensor', dist.Normal(temp, 1.0))
  return (temp, sensor)

print(model2())

In [ ]:
pyro.render_model(model2, render_distributions=True)